In [3]:
import sys
import os

sys.path

# sys.path.insert(1, os.path.join(os.getcwd()  , '..'))

from app.init import init_llm

init_llm(2)



Init AIPROXY env


<module 'openai' from '/Users/lingoace/Documents/GitHub/llm-101/venv/lib/python3.10/site-packages/openai/__init__.py'>

## Create a custom prompt template
docs: https://python.langchain.com/en/latest/modules/prompts/prompt_templates/examples/custom_prompt_template.html

In [4]:
from langchain.prompts import StringPromptTemplate
from pydantic import BaseModel, validator
import inspect

def get_source_code(fn_name):
    return inspect.getsource(fn_name)


class FunctionExplainerPromptTeamplte(StringPromptTemplate, BaseModel):
    """A custom promt template that takes in the function name as input, and formats the prompt template to
    provide a description of the function.

    Args:
        StringPromptTemplate (_type_): _description_
        BaseModel (_type_): _description_
    """
    
    @validator("input_variables")
    def validate_input_variables(cls, v):
        if len(v) != 1 or 'fn_name' not in v:
            raise ValueError('input_variables must be a list of length 1, containing the variable fn_name')
        return v
    
    def format(self, **kwargs) -> str:
        source_code = get_source_code(kwargs['fn_name'])
        
        prompt = f"""
            Give the function name and source code, explain what the function does.
            Function Name: {kwargs['fn_name'].__name__}
            Source Code: 
            ```
            {source_code}
            ```
            Explanation:
        """
        return prompt
    
    def _prompt_type(self) -> str:
        return 'function-explainer'

In [6]:
import requests

def req_api(url: str, count: int):
    return requests.get(url, count=count)    

fn_explainer = FunctionExplainerPromptTeamplte(input_variables=['fn_name'])

prompt = fn_explainer.format(fn_name=req_api)
print(prompt)


            Give the function name and source code, explain what the function does.
            Function Name: req_api
            Source Code: 
            ```
            def req_api(url: str, count: int):
    return requests.get(url, count=count)    

            ```
            Explanation:
        


In [10]:
from langchain.llms import OpenAI

llm = OpenAI(max_tokens=100, temperature=0.5)
print(llm(prompt))


            This function is called req_api and it takes two parameters: url (of type string) and count (of type int). The function uses the requests library to make an HTTP GET request to the given url parameter with the given count parameter. The function then returns the response from the request.
